# Lab 02: Introduction to Text Preprocessing & the Spacy Toolkit

### Objectives:
1. Get familiar with basic text preprocessing pipelines
2. Get familiar with regular expressions, and the `re` package in Python
3. Evaluate the lexical diversity of the data in each category within the 20 News Groups Dataset
4. Use normalized BOW features to evaluate text similarity using the KL-divergence

### Required Reading:

1. https://universaldependencies.org/u/pos/
2. https://spacy.io/api/annotation#pos-tagging
3. https://spacy.io/api/annotation#dependency-parsing

# Part I: Introduction to Spacy

### Download Spacy's base English language *pipeline* components

``$ spacy load en_core_web_sm``

What is a Spacy *pipeline*? A Spacy pipeline is an extensible tool that streamlines many of the common tasks in NLP, such as tokenization, part-of-speech tagging, named entity recognition, stemming, lemmatizing, and parsing. It also has custom pipeline components specifically for transformers. It is built for production use; much thought and care has gone into its API and implementation. You can actually configure Spacy to use some of the statistical models that we will discover in this class; for now we're just going to cover some of the basics.

In [ ]:
import spacy

pipeline = spacy.load('en_core_web_sm')

### Download the 20 News Groups dataset using the sklearn package

This data consists of news articles from 20 different categories. 

In [ ]:
from sklearn.datasets import fetch_20newsgroups
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

ng_train = fetch_20newsgroups(subset='train')
ng_test = fetch_20newsgroups(subset='test')
ng_train.target_names

### Get the number of training & test examples

In [ ]:
len(ng_train.data), len(ng_test.data)

### Take a peek at the first document and its label

In [ ]:
ng_train.data[0]

In [ ]:
label_idx = ng_train.target[0]
ng_train.target_names[label_idx]

### Evaluate Spacy's recognition of entities, POS

In [ ]:
from pprint import pprint

doc = pipeline(ng_train.data[0])
for i, token in enumerate(doc):
    pprint({"text": token.text,
            "lemma": token.lemma_,
            "POS": token.pos_,
            "tag": token.tag_,
            "dep": token.dep_,
            "shape": token.shape_,
            "is_alpha": token.is_alpha,
            "is_stop": token.is_stop})
    if i == 3:
        break

### Let's define a preprocessing function that cleans our data

You'll notice that even the lemmatized text contains meaningless tokens. In the real world you're never going to get around having to do some feature engineering. In NLP this often means writing some regexes to transform text into a usable format. This has become less important in the deep learning era, but applying domain specific knowledge is always beneficial. In the case of this dataset, we have text that originated in news feeds, some of which is messy. There are email and url addresses, grammatical errors, and a lot puntuation and uninformative characters (e.g., the newline character `\n`). Below is a function that does some very basic regex (regular expression) matching to strip out emails, urls, punctuation, and other junk.

In [ ]:
import re
from spacy.language import Language


# http://emailregex.com/
email_re = r"""(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])"""

# replace = [ (pattern-to-replace, replacement),  ...]
replace = [
    (r"<a[^>]*>(.*?)</a>", r"\1"),  # Matches most URLs
    (email_re, "email"),            # Matches emails
    (r"[\t\n\r\*\.]", " "),         # Punctuation and other junk
    (r"\s+", " ")                   # Stips extra whitespace
]

train_text = ng_train.data
test_text = ng_test.data
for repl in replace:
    train_text = [re.sub(repl[0], repl[1], text) for text in train_text]
    test_text = [re.sub(repl[0], repl[1], text) for text in test_text]

@Language.component("ng20")
def ng20_preprocess(doc):
    tokens = [tokens.lower_ for tokens in doc]
    tokens = [tokens for tokens in doc if tokens.is_stop == False]
    tokens = [tokens for tokens in tokens if tokens.is_punct == False]
    final_token = [token.lemma_ for token in tokens]
    return " ".join(final_token)

pipeline.add_pipe("ng20")

#### Peek at our processing pipeline

In [ ]:
pipeline.analyze_pipes(pretty=True)

### Now pass each training and test document through the pipeline

In [ ]:
docs_train = [pipeline(doc) for doc in train_text[:500]]
docs_test = [pipeline(doc) for doc in test_text[:500]]

### Let's look at that first document following this transformation and compare it to the original text

In [ ]:
ng_train.data[0]

In [ ]:
docs_train[0]

# Part II: Lexical diversity

Sometimes it's useful to understand how diverse is the language in some body of text. Once simple heuristic to evaluate diversity is as follows: 

$$ lexical\_diversity = \frac{ len(set(all\_words\_in\_doc)) }{ len(doc) }$$

Find the set of all words observed in the document, and divide it by the number of total words in the document. Let's use this to evalute the diversity of each category in the 20NG dataset.

### (5 pts) Task I: 
In the cell below, compute the diversity of each category in the 20NG dataset using the above heuristic

In [ ]:
# Your code goes here

### Explain these scores: 

1. Is this result real or an artifact of some underlying problem with our data? 
2. What might you do to better evaluate lexical diversity on this data using this scoring function?
3. Is this heuristic a good metric for lexical diversity in general?

### Entropy 
Entropy is another, perhaps more principled, way by which we can evaluate how diverse, or varied, is a piece of text. Recall the definition of Entropy, $H(p(x))$:

$$ H(p(x)) = \sum_{i=1}^{N} -p(x_{i}) \log p(x_{i}) $$

In the Bag-of-Words (BOW) feature representation of a document, each document is represented by a word count vector, ${x}_{i} \in \mathbb{R}^{N}$ where $N$ is the cardinality of the set of words in the document.

### (5 pts) Task II:
In order to compute an entropy from this representation, you'll first need to convert those count vectors into probability distributions. Then compute the entropy of the word distributions aggregated over each news category.

In [ ]:
# Your code goes here

### Explain this result

1. What does it mean for a distribution to have high or low entropy?
2. Do these scores make intuitive sense? Any more or less so than the heuristic from Task I?
2. Is entropy a good metric for evaluating lexical diversity in general?

# Part III: Document Similarity

Throughout this course we will discuss the notion of *similarity* between texts and explore ways to measure it. This is a critical component of search and recommender systems. One such approach involves measuring how *close* two word distributions are using the notion divergence, which we discussed in the first lecture.

### (10 pts) Task III

Using the definition below, compute the KL-divergence, $K_{DL}$, between the word distributions in each category. This will result in a $K \times K$ matrix of divergence values.

$$ D_{KL}(P||Q) = \sum_{i=1}^{N} \log \frac{P(x_{i})}{Q(x_{i})} $$

In [ ]:
# Your code goes here

### Explain this result

1. What does it mean for two distributions to have high or low divergence?
2. Do these similarity scores make sense intuitively?
3. Is the resultant $K \times K$ matrix symmetric? Why is this the case?
2. Is $D_{KL}$ a good measure of the similarity between two distributions in general? 